In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras import datasets, layers, models
import os, random

from modules.painting import painting
from modules.database import database as db
# from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import backend as K

#### Control Variables

In [ ]:
# setting path variables for Working Directory and folder to save and load
#   models from.
wd = os.getcwd()
WD_PATH =  os.path.abspath(wd)
PATH_TRAINING = os.path.join(WD_PATH, "model_training")

# # initializing the database object
gallery = db()

# ### How many pictures should be used per artist
TOTAL_PAINTINGS = 3971
SIZE_TRAINING = 190
SIZE_TESTING = 38
SIZE_UNUSED = 3971 - ((SIZE_TESTING + SIZE_TRAINING) * 10 )
PIXEL_SIZE = 100

#### Loading data from DB

In [ ]:
## creating arrays to hold the pictures taken from the DB
testing_images = np.zeros((10*SIZE_TESTING,PIXEL_SIZE,PIXEL_SIZE,3))
index_testing = 0
skip_count_testing = 0
training_images = np.zeros((10*SIZE_TRAINING,PIXEL_SIZE,PIXEL_SIZE,3))
index_training = 0
skip_count_training = 0
# creating the arrays to hold labels. In this case they are the artist ids.
# testing_labels = np.zeros((10*SIZE_TESTING,1),dtype=int)
# training_labels = np.zeros((10*SIZE_TRAINING,1),dtype=int)
testing_labels = np.array([0]*SIZE_TESTING*10,dtype=int)
training_labels = np.array([0]*SIZE_TRAINING*10,dtype=int)
unused_paintings = np.array([0]*SIZE_UNUSED,dtype=int)


# filling the arrays with picture arrays. They will be resized according
# to the pixel_size value
unused_index = 0
for i in range(1,11):
    ids = gallery.get_paintingids_from_artist(i)
    # random.seed(1983)
    random.shuffle(ids)
    ids_training = ids[ : SIZE_TRAINING]
    ids_testing = ids[SIZE_TRAINING : SIZE_TRAINING + SIZE_TESTING]
    ids_unused = ids[SIZE_TRAINING + SIZE_TESTING : ]
    for l, f in zip(range(unused_index,unused_index+len(ids_unused))
                    , ids_unused):
        unused_paintings[l] = f[0]
    unused_index += len(ids_unused)
    for j in ids_testing:
        temp_p = painting("local DB", id=j[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            testing_images[index_testing] = temp_p_res
            testing_labels[index_testing] = temp_p.artist_id-1
            # testing_labels[index_testing] = [temp_p.artist_id-1,]
            index_testing += 1
        else:
            skip_count_testing += 1
    # print("Testing images status")
    # print(index_testing)
    # print(skip_count_testing)
    for k in ids_training:
        temp_p = painting("local DB", id=k[0])
        temp_p_res = cv.resize(temp_p.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
        if temp_p_res.shape == (PIXEL_SIZE,PIXEL_SIZE,3):
            training_images[index_training] = temp_p_res
            training_labels[index_training] = temp_p.artist_id-1
            # training_labels[index_training] = [temp_p.artist_id-1,]
            index_training += 1
        else:
            skip_count_training += 1
    # print("Training images status")
    # print(index_training)
    # print(skip_count_training)

# ## dropping the last few array positions of testing images, which where 
# ## not filled.
testing_images = testing_images[:index_testing,:,:,:]
testing_labels = testing_labels[:index_testing]

# # the pixels on an image are rescaled from 0-255 to 0-1 
training_images, testing_images = training_images/255, testing_images/255 

# defining labels in a list
class_names = [i[1] for i in gallery.get_all_artists()]

#### Checking data shapes

In [ ]:
print(training_images.shape)
print(testing_images.shape)
print(training_labels.shape)
print(testing_labels.shape)
print(type(training_labels[185:200]))

In [ ]:
# print(np.unique(training_labels))
# print(type(training_labels[21]))
# print(training_labels[21].shape)
# print(training_labels[500])
# print(training_labels[:,0].shape)
# print(type(training_labels[:,0]))

In [ ]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=(PIXEL_SIZE,PIXEL_SIZE,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
 
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

In [ ]:
## different version
model = Sequential()
model.add(Conv2D(64,(3,3),padding='same',input_shape=(PIXEL_SIZE, PIXEL_SIZE , 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128,(3 , 3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
# model. add ( Convolutional -Schichten (64 ,(5 ,5 ), padding = 'same' ))
# model. add ( BatchNormalization ())
# model. add ( Activation ('ReLU' ))
# model. add ( MaxPooling2D (( 4 ,4 )))

# model .add(Convolutional-Schichten(128 ,(7 ,7 ), padding = ' same '))
# model .add(BatchNormalization ())
# model .add ( Activation ('ReLU' ))

model.add(Flatten ())
  
## fully connected layer
for i in range(0):
    units=256//(i+1)
    if units<32:
        break
    
    if i ==0:
        input_dim=model.output_shape[1]
        
    else:
        input_dim=units
        
    print(input_dim)
    
    # Add the Dense Layer along with activation and batch normalization
    
    dense_layer=Dense(units=units,input_dim=input_dim,)
    
    # Add the Leaky ReLU activation layer
    
   
    
    
    

dense_layer=Dense(units=64,input_dim=model.output_shape[1])
activation_layer=Activation('relu')
batch_norm_layer = BatchNormalization()

model.add(dense_layer)
model.add(activation_layer)
model.add(batch_norm_layer)

## Add Dropout Layer
dropout_fraction=0.5
dropout_layer=Dropout(dropout_fraction)

## Add the dense layer for final output
dense_output=Dense(units=1, input_dim=model.output_shape[1], activation='sigmoid')
activation_output = Activation('sigmoid')

model.add(dropout_layer)
model.add(dense_output)

In [ ]:
opt = Adam(lr=0.0001) # learning rate

# Compile the Model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
history = model.fit(training_images,training_labels,batch_size=1,
                    epochs=10,
                    validation_data=(testing_images,testing_labels))

In [ ]:
loss,accuracy = model.evaluate(testing_images, testing_labels)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
model.save(os.path.join(PATH_TRAINING,"image_classifier_BvDtest.model"))

In [ ]:
model = models.load_model(os.path.join(PATH_TRAINING,"image_classifier_BvDtest.model"))

#### check one prediction

In [ ]:
test_painting = painting("local DB",random.choice(unused_paintings))
print(test_painting.id)
test_ndarray = cv.resize(test_painting.ndarray, dsize=(PIXEL_SIZE,PIXEL_SIZE)
                               ,interpolation=cv.INTER_CUBIC)
test_ndarray = test_ndarray/255
temp_array = np.zeros((1,PIXEL_SIZE,PIXEL_SIZE,3))
temp_array[0] = test_ndarray
print(temp_array.shape)
print(test_ndarray.shape)
prediction = model.predict(temp_array)
index = np.argmax(prediction)
print(f"Prediction is {class_names[index]}")
imgplot = plt.imshow(test_painting.ndarray)
plt.show()